In [33]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

# 10 Vector Semantics for tutorial

## Latent Semantic Analysis

In this notebook, we will use a toy example to illustrate what is the vector semantics. The key idea is that how to build the co-occurrence matrix between words and **context**.

## Agenda

1. Word-document matrix
    
2. Word-word matrix

3. Reduced SVD for word-word matrix: sparse  to dense

    C = U_r\*S_r\*VT_r that r is the rank of the matrix.


4. Truncated SVD for word-word matrix: sparse to dense and low dimensional

    C = U_k\*S_k\*VT_k that k is less than the rank of the matrix.  

## Part 1 Word-document Matrix

In [93]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [71]:
# define the corpus
documents = ["I eat an apple every day", 
            "I eat an orange every day", 
             "I like driving my car to work"]

In [72]:
bow = CountVectorizer()
sparse_fea = bow.fit_transform(documents)

In [73]:
hh = sparse_fea.todense()

In [74]:
fea_names = bow.get_feature_names()

In [75]:
# show the document-term matrix
df_raw = pd.DataFrame(sparse_fea.todense(), columns=fea_names)
display(df_raw)

an  apple  car  day  driving  eat  every  like  my  orange  to  work
0   1      1    0    1        0    1      1     0   0       0   0     0
1   1      0    0    1        0    1      1     0   0       1   0     0
2   0      0    1    0        1    0      0     1   1       0   1     1

In [76]:
# show the term-document matrix
df_raw = pd.DataFrame(sparse_fea.todense().T, index=fea_names, columns=['0', '1', '2'])
display(df_raw)

0  1  2
an       1  1  0
apple    1  0  0
car      0  0  1
day      1  1  0
driving  0  0  1
eat      1  1  0
every    1  1  0
like     0  0  1
my       0  0  1
orange   0  1  0

#### Compute the similarity between docs "apple" and "orange"

In [77]:
vec_doc4 = sparse_fea.todense()[1]
vec_doc5 = sparse_fea.todense()[-1]
print(cosine_similarity(vec_doc4, vec_doc5))

[[0.]]


## Part 2 Word-Word Matrix

The dimensionality of the text data can be transformed to the rank of the raw matrix. Here is the rank is 5.

In [79]:
import numpy as np
import pandas as pd
from itertools import combinations
from collections import Counter

# build vocab
vocab = list(set((' '.join(documents)).split(' ')))
print('Vocabulary:\n',vocab,'\n')

# tokenize each document
token_sent_list = [doc.split(' ') for doc in documents]
print('Each sentence in token form:\n',token_sent_list,'\n')

co_occ = {ii:Counter({jj:0 for jj in vocab if jj!=ii}) for ii in vocab}
# windows size
k=3

# build the co-occurrence dictionary
for sen in token_sent_list:
    for ii in range(len(sen)):
        if ii < k:
            c = Counter(sen[0:ii+k+1])
            del c[sen[ii]]
            co_occ[sen[ii]] = co_occ[sen[ii]] + c
        elif ii > len(sen)-(k+1):
            c = Counter(sen[ii-k::])
            del c[sen[ii]]
            co_occ[sen[ii]] = co_occ[sen[ii]] + c
        else:
            c = Counter(sen[ii-k:ii+k+1])
            del c[sen[ii]]
            co_occ[sen[ii]] = co_occ[sen[ii]] + c

Vocabulary:
 ['like', 'an', 'to', 'my', 'driving', 'apple', 'orange', 'work', 'every', 'car', 'I', 'eat', 'day'] 

Each sentence in token form:
 [['I', 'eat', 'an', 'apple', 'every', 'day'], ['I', 'eat', 'an', 'orange', 'every', 'day'], ['I', 'like', 'driving', 'my', 'car', 'to', 'work']] 



In [80]:
# show the co-occurrence matrix
co_occ = {ii:dict(co_occ[ii]) for ii in vocab}
mat = np.zeros((len(vocab), len(vocab)))
for l_key in co_occ:
    left_idx = vocab.index(l_key)
    
    for r_key in co_occ[l_key]:
        right_idx = vocab.index(r_key)
        mat[left_idx, right_idx] = co_occ[l_key][r_key]
mat_df = pd.DataFrame(mat)
mat_df.index = vocab
mat_df.columns = vocab
display(mat_df)

like   an   to   my  driving  apple  orange  work  every  car    I  \
like      0.0  0.0  0.0  1.0      1.0    0.0     0.0   0.0    0.0  1.0  1.0   
an        0.0  0.0  0.0  0.0      0.0    1.0     1.0   0.0    2.0  0.0  2.0   
to        0.0  0.0  0.0  1.0      1.0    0.0     0.0   1.0    0.0  1.0  0.0   
my        1.0  0.0  1.0  0.0      1.0    0.0     0.0   1.0    0.0  1.0  1.0   
driving   1.0  0.0  1.0  1.0      0.0    0.0     0.0   0.0    0.0  1.0  1.0   
apple     0.0  1.0  0.0  0.0      0.0    0.0     0.0   0.0    1.0  0.0  1.0   
orange    0.0  1.0  0.0  0.0      0.0    0.0     0.0   0.0    1.0  0.0  1.0   
work      0.0  0.0  1.0  1.0      0.0    0.0     0.0   0.0    0.0  1.0  0.0   
every     0.0  2.0  0.0  0.0      0.0    1.0     1.0   0.0    0.0  0.0  0.0   
car       1.0  0.0  1.0  1.0      1.0    0.0     0.0   1.0    0.0  0.0  0.0   
I         1.0  2.0  0.0  1.0      1.0    1.0     1.0   0.0    0.0  0.0  0.0   
eat       0.0  2.0  0.0  0.0      0.0    1.0     1.0   0.0    2.0  0.0  2.0   
day       0.0  2.0  0.0  0.0      0.0    1.0     1.0   0.0    2.0  0.0  0.0   

         eat  day  
like     0.0  0.0  
an       2.0  2.0  
to       0.0  0.0  
my       0.0  0.0  
driving  0.0  0.0  
apple    1.0  1.0  
orange   1.0  1.0  
work     0.0  0.0  
every    2.0  2.0  
car      0.0  0.0  
I        2.0  0.0  
eat      0.0  0.0  
day      0.0  0.0

In [94]:
# compute the distance
vec_apple = mat[vocab.index('apple')].reshape(1, -1)
vec_orange = mat[vocab.index('orange')].reshape(1, -1)
vec_car = mat[vocab.index('car')].reshape(1, -1)
print('cosine scores between apple and orange vectors')
print(cosine_similarity(vec_apple, vec_orange))
print('cosine scores between apple and car vectors')
print(cosine_similarity(vec_apple, vec_car))

cosine scores between apple and orange vectors
[[1.]]
cosine scores between apple and car vectors
[[0.]]


The vector is still very sparse. Then, we are going to use SVD to transform the original vector into low and dense one

## Part 3 Reduced SVD

The dimensionality of the text data can be transformed to the rank of the raw matrix. Here is the rank is 13, i.e., the size of vocab.

In [82]:
from scipy import linalg
import numpy as np
U, s, Vh = linalg.svd(mat, full_matrices=False)

In [83]:
dim_names = ['dim{}'.format(idx) for idx in range(len(vocab))]

In [95]:
# Show the matrix U
U_df = pd.DataFrame(U, index=vocab, columns=dim_names)
display(U_df)

dim0      dim1      dim2      dim3      dim4      dim5      dim6  \
like    -0.077212  0.343247 -0.123436  0.037926 -0.421132  0.418296  0.259562   
an      -0.491788 -0.091238 -0.335890  0.541625  0.147133  0.173819 -0.016109   
to      -0.030940  0.365767  0.055172 -0.018568 -0.103278  0.610431 -0.329497   
my      -0.083492  0.465357 -0.144095  0.045133  0.440313 -0.146075 -0.022304   
driving -0.080940  0.411456 -0.146140  0.046549 -0.272527 -0.546514  0.128773   
apple   -0.275520 -0.054126 -0.098831  0.145046 -0.032287 -0.050455 -0.011252   
orange  -0.275520 -0.054126 -0.098831  0.145046 -0.032287 -0.050455 -0.011252   
work    -0.021188  0.289037 -0.004970  0.003050 -0.495413 -0.253355 -0.380611   
every   -0.414589 -0.151214  0.513992  0.237983 -0.137082 -0.154744 -0.308413   
car     -0.040242  0.429776  0.105969 -0.036181  0.502682 -0.050703 -0.226468   
I       -0.358978  0.190798  0.607645 -0.175093  0.043345  0.060341  0.514355   
eat     -0.422154 -0.048496 -0.400540 -0.475415 -0.024908 -0.035478  0.235080   
day     -0.323804 -0.135970 -0.046223 -0.586470  0.026238  0.038438 -0.442008   

             dim7      dim8      dim9     dim10     dim11         dim12  
like     0.121355 -0.047526  0.162807 -0.403278 -0.483450  2.155778e-16  
an      -0.518700 -0.131090 -0.070519 -0.003374  0.017344 -1.149028e-16  
to       0.107459 -0.006569 -0.040317  0.541099  0.252991 -1.584945e-16  
my       0.382608 -0.443865 -0.395969 -0.192186  0.028792  1.391469e-16  
driving -0.135582  0.007271  0.023431  0.547514 -0.303878  1.296472e-17  
apple    0.360764  0.481197 -0.157151 -0.004652  0.014827 -7.071068e-01  
orange   0.360764  0.481197 -0.157151 -0.004652  0.014827  7.071068e-01  
work    -0.216722  0.054084 -0.121318 -0.420966  0.468682 -1.821405e-16  
every    0.294096 -0.358219  0.341631  0.014517 -0.121797  4.242162e-16  
car     -0.182704  0.389867  0.522511 -0.167521 -0.090160  2.507791e-16  
I       -0.236430  0.099950 -0.265685 -0.014037  0.167585 -4.028907e-16  
eat      0.141402 -0.164013  0.426951  0.035025  0.381356 -3.152512e-17  
day     -0.192014  0.014488 -0.320685 -0.030126 -0.438426  6.812404e-17

In [85]:
# Show the matrix S
S_df = pd.DataFrame(np.diag(s), columns=dim_names, index=dim_names)
display(S_df)

dim0      dim1      dim2      dim3      dim4      dim5      dim6  \
dim0   7.300072  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
dim1   0.000000  4.362418  0.000000  0.000000  0.000000  0.000000  0.000000   
dim2   0.000000  0.000000  3.429951  0.000000  0.000000  0.000000  0.000000   
dim3   0.000000  0.000000  0.000000  3.153287  0.000000  0.000000  0.000000   
dim4   0.000000  0.000000  0.000000  0.000000  1.694984  0.000000  0.000000   
dim5   0.000000  0.000000  0.000000  0.000000  0.000000  1.632695  0.000000   
dim6   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  1.519317   
dim7   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
dim8   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
dim9   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
dim10  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
dim11  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
dim12  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

          dim7      dim8      dim9     dim10     dim11         dim12  
dim0   0.00000  0.000000  0.000000  0.000000  0.000000  0.000000e+00  
dim1   0.00000  0.000000  0.000000  0.000000  0.000000  0.000000e+00  
dim2   0.00000  0.000000  0.000000  0.000000  0.000000  0.000000e+00  
dim3   0.00000  0.000000  0.000000  0.000000  0.000000  0.000000e+00  
dim4   0.00000  0.000000  0.000000  0.000000  0.000000  0.000000e+00  
dim5   0.00000  0.000000  0.000000  0.000000  0.000000  0.000000e+00  
dim6   0.00000  0.000000  0.000000  0.000000  0.000000  0.000000e+00  
dim7   1.41823  0.000000  0.000000  0.000000  0.000000  0.000000e+00  
dim8   0.00000  1.119883  0.000000  0.000000  0.000000  0.000000e+00  
dim9   0.00000  0.000000  0.710734  0.000000  0.000000  0.000000e+00  
dim10  0.00000  0.000000  0.000000  0.430897  0.000000  0.000000e+00  
dim11  0.00000  0.000000  0.000000  0.000000  0.408854  0.000000e+00  
dim12  0.00000  0.000000  0.000000  0.000000  0.000000  3.103053e-17

In [86]:
# Show the matrix VT
VT_df = pd.DataFrame(Vh, columns=vocab, index=dim_names)
display(VT_df)

like            an            to            my       driving  \
dim0  -7.721189e-02 -4.917883e-01 -3.093971e-02 -8.349228e-02 -8.093953e-02   
dim1   3.432471e-01 -9.123794e-02  3.657665e-01  4.653569e-01  4.114564e-01   
dim2   1.234356e-01  3.358902e-01 -5.517168e-02  1.440953e-01  1.461405e-01   
dim3  -3.792617e-02 -5.416252e-01  1.856816e-02 -4.513307e-02 -4.654934e-02   
dim4   4.211321e-01 -1.471333e-01  1.032779e-01 -4.403129e-01  2.725274e-01   
dim5  -4.182965e-01 -1.738194e-01 -6.104308e-01  1.460752e-01  5.465136e-01   
dim6   2.595617e-01 -1.610942e-02 -3.294971e-01 -2.230386e-02  1.287734e-01   
dim7  -1.213547e-01  5.186996e-01 -1.074586e-01 -3.826076e-01  1.355820e-01   
dim8   4.752596e-02  1.310900e-01  6.569245e-03  4.438655e-01 -7.271231e-03   
dim9  -1.628070e-01  7.051856e-02  4.031679e-02  3.959689e-01 -2.343100e-02   
dim10  4.032778e-01  3.374445e-03 -5.410989e-01  1.921855e-01 -5.475141e-01   
dim11 -4.834499e-01  1.734394e-02  2.529911e-01  2.879163e-02 -3.038776e-01   
dim12  2.170146e-16  1.097664e-16 -7.945451e-18 -8.488686e-17  3.785703e-16   

          apple    orange          work         every           car  \
dim0  -0.275520 -0.275520 -2.118805e-02 -4.145893e-01 -4.024227e-02   
dim1  -0.054126 -0.054126  2.890369e-01 -1.512144e-01  4.297763e-01   
dim2   0.098831  0.098831  4.969638e-03 -5.139915e-01 -1.059693e-01   
dim3  -0.145046 -0.145046 -3.049646e-03 -2.379832e-01  3.618131e-02   
dim4   0.032287  0.032287  4.954127e-01  1.370824e-01 -5.026815e-01   
dim5   0.050455  0.050455  2.533555e-01  1.547437e-01  5.070328e-02   
dim6  -0.011252 -0.011252 -3.806115e-01 -3.084135e-01 -2.264685e-01   
dim7  -0.360764 -0.360764  2.167223e-01 -2.940959e-01  1.827042e-01   
dim8  -0.481197 -0.481197 -5.408356e-02  3.582195e-01 -3.898674e-01   
dim9   0.157151  0.157151  1.213183e-01 -3.416309e-01 -5.225106e-01   
dim10  0.004652  0.004652  4.209656e-01 -1.451707e-02  1.675207e-01   
dim11  0.014827  0.014827  4.686824e-01 -1.217973e-01 -9.015999e-02   
dim12 -0.707107  0.707107 -4.228841e-16  2.446230e-16  5.655008e-17   

                  I           eat           day  
dim0  -3.589782e-01 -4.221535e-01 -3.238043e-01  
dim1   1.907977e-01 -4.849639e-02 -1.359698e-01  
dim2  -6.076447e-01  4.005395e-01  4.622271e-02  
dim3   1.750932e-01  4.754155e-01  5.864699e-01  
dim4  -4.334516e-02  2.490760e-02 -2.623761e-02  
dim5  -6.034137e-02  3.547826e-02 -3.843800e-02  
dim6   5.143553e-01  2.350800e-01 -4.420076e-01  
dim7   2.364301e-01 -1.414021e-01  1.920138e-01  
dim8  -9.995029e-02  1.640132e-01 -1.448814e-02  
dim9   2.656852e-01 -4.269511e-01  3.206854e-01  
dim10  1.403682e-02 -3.502519e-02  3.012650e-02  
dim11  1.675855e-01  3.813557e-01 -4.384256e-01  
dim12 -2.234455e-16 -2.473338e-16  2.193178e-16

### Get the new representation for the words

compared to the previous one, it is dense. 

In [87]:
dense_matrix = U*s
RSVD_df = pd.DataFrame(dense_matrix, index=vocab, columns=dim_names)
display(RSVD_df)

dim0      dim1      dim2      dim3      dim4      dim5      dim6  \
like    -0.563652  1.497387 -0.423378  0.119592 -0.713812  0.682951  0.394356   
an      -3.590090 -0.398018 -1.152087  1.707900  0.249389  0.283794 -0.024475   
to      -0.225862  1.595626  0.189236 -0.058551 -0.175054  0.996648 -0.500610   
my      -0.609500  2.030081 -0.494240  0.142318  0.746323 -0.238496 -0.033887   
driving -0.590864  1.794945 -0.501255  0.146783 -0.461930 -0.892290  0.195648   
apple   -2.011314 -0.236121 -0.338984  0.457370 -0.054726 -0.082377 -0.017095   
orange  -2.011314 -0.236121 -0.338984  0.457370 -0.054726 -0.082377 -0.017095   
work    -0.154674  1.260900 -0.017046  0.009616 -0.839717 -0.413652 -0.578269   
every   -3.026532 -0.659660  1.762966  0.750429 -0.232352 -0.252649 -0.468578   
car     -0.293771  1.874864  0.363469 -0.114090  0.852037 -0.082783 -0.344077   
I       -2.620567  0.832339  2.084192 -0.552119  0.073469  0.098519  0.781469   
eat     -3.081751 -0.211562 -1.373831 -1.499121 -0.042218 -0.057925  0.357161   
day     -2.363795 -0.593157 -0.158542 -1.849308  0.044472  0.062758 -0.671550   

             dim7      dim8      dim9     dim10     dim11         dim12  
like     0.172109 -0.053224  0.115712 -0.173771 -0.197660  6.689493e-33  
an      -0.735635 -0.146805 -0.050120 -0.001454  0.007091 -3.565497e-33  
to       0.152401 -0.007357 -0.028655  0.233158  0.103436 -4.918169e-33  
my       0.542625 -0.497077 -0.281428 -0.082812  0.011772  4.317802e-33  
driving -0.192286  0.008143  0.016653  0.235922 -0.124242  4.023020e-34  
apple    0.511646  0.538884 -0.111693 -0.002004  0.006062 -2.194190e-17  
orange   0.511646  0.538884 -0.111693 -0.002004  0.006062  2.194190e-17  
work    -0.307362  0.060567 -0.086225 -0.181393  0.191623 -5.651917e-33  
every    0.417095 -0.401164  0.242809  0.006255 -0.049797  1.316365e-32  
car     -0.259116  0.436606  0.371366 -0.072184 -0.036862  7.781809e-33  
I       -0.335312  0.111933 -0.188831 -0.006048  0.068518 -1.250191e-32  
eat      0.200541 -0.183676  0.303448  0.015092  0.155919 -9.782413e-34  
day     -0.272320  0.016225 -0.227922 -0.012981 -0.179252  2.113925e-33

## Part 4 Truncated SVD for text data

1. set the small singular values in S to zero, i.e., ignore the unimportant dimensions
2. Try the latent dimension is 5

In [88]:
# set the singular values to zero 
s[5: ] = 0

In [89]:
new_Sdf = pd.DataFrame(np.diag(s), columns=dim_names, index=dim_names)
display(new_Sdf)

dim0      dim1      dim2      dim3      dim4  dim5  dim6  dim7  \
dim0   7.300072  0.000000  0.000000  0.000000  0.000000   0.0   0.0   0.0   
dim1   0.000000  4.362418  0.000000  0.000000  0.000000   0.0   0.0   0.0   
dim2   0.000000  0.000000  3.429951  0.000000  0.000000   0.0   0.0   0.0   
dim3   0.000000  0.000000  0.000000  3.153287  0.000000   0.0   0.0   0.0   
dim4   0.000000  0.000000  0.000000  0.000000  1.694984   0.0   0.0   0.0   
dim5   0.000000  0.000000  0.000000  0.000000  0.000000   0.0   0.0   0.0   
dim6   0.000000  0.000000  0.000000  0.000000  0.000000   0.0   0.0   0.0   
dim7   0.000000  0.000000  0.000000  0.000000  0.000000   0.0   0.0   0.0   
dim8   0.000000  0.000000  0.000000  0.000000  0.000000   0.0   0.0   0.0   
dim9   0.000000  0.000000  0.000000  0.000000  0.000000   0.0   0.0   0.0   
dim10  0.000000  0.000000  0.000000  0.000000  0.000000   0.0   0.0   0.0   
dim11  0.000000  0.000000  0.000000  0.000000  0.000000   0.0   0.0   0.0   
dim12  0.000000  0.000000  0.000000  0.000000  0.000000   0.0   0.0   0.0   

       dim8  dim9  dim10  dim11  dim12  
dim0    0.0   0.0    0.0    0.0    0.0  
dim1    0.0   0.0    0.0    0.0    0.0  
dim2    0.0   0.0    0.0    0.0    0.0  
dim3    0.0   0.0    0.0    0.0    0.0  
dim4    0.0   0.0    0.0    0.0    0.0  
dim5    0.0   0.0    0.0    0.0    0.0  
dim6    0.0   0.0    0.0    0.0    0.0  
dim7    0.0   0.0    0.0    0.0    0.0  
dim8    0.0   0.0    0.0    0.0    0.0  
dim9    0.0   0.0    0.0    0.0    0.0  
dim10   0.0   0.0    0.0    0.0    0.0  
dim11   0.0   0.0    0.0    0.0    0.0  
dim12   0.0   0.0    0.0    0.0    0.0

In [90]:
lowdim_dense_matrix = U*s

In [91]:
TSVD_df = pd.DataFrame(lowdim_dense_matrix, index=vocab, columns=dim_names)
display(TSVD_df)

dim0      dim1      dim2      dim3      dim4  dim5  dim6  dim7  \
like    -0.563652  1.497387 -0.423378  0.119592 -0.713812   0.0   0.0   0.0   
an      -3.590090 -0.398018 -1.152087  1.707900  0.249389   0.0  -0.0  -0.0   
to      -0.225862  1.595626  0.189236 -0.058551 -0.175054   0.0  -0.0   0.0   
my      -0.609500  2.030081 -0.494240  0.142318  0.746323  -0.0  -0.0   0.0   
driving -0.590864  1.794945 -0.501255  0.146783 -0.461930  -0.0   0.0  -0.0   
apple   -2.011314 -0.236121 -0.338984  0.457370 -0.054726  -0.0  -0.0   0.0   
orange  -2.011314 -0.236121 -0.338984  0.457370 -0.054726  -0.0  -0.0   0.0   
work    -0.154674  1.260900 -0.017046  0.009616 -0.839717  -0.0  -0.0  -0.0   
every   -3.026532 -0.659660  1.762966  0.750429 -0.232352  -0.0  -0.0   0.0   
car     -0.293771  1.874864  0.363469 -0.114090  0.852037  -0.0  -0.0  -0.0   
I       -2.620567  0.832339  2.084192 -0.552119  0.073469   0.0   0.0  -0.0   
eat     -3.081751 -0.211562 -1.373831 -1.499121 -0.042218  -0.0   0.0   0.0   
day     -2.363795 -0.593157 -0.158542 -1.849308  0.044472   0.0  -0.0  -0.0   

         dim8  dim9  dim10  dim11  dim12  
like     -0.0   0.0   -0.0   -0.0    0.0  
an       -0.0  -0.0   -0.0    0.0   -0.0  
to       -0.0  -0.0    0.0    0.0   -0.0  
my       -0.0  -0.0   -0.0    0.0    0.0  
driving   0.0   0.0    0.0   -0.0    0.0  
apple     0.0  -0.0   -0.0    0.0   -0.0  
orange    0.0  -0.0   -0.0    0.0    0.0  
work      0.0  -0.0   -0.0    0.0   -0.0  
every    -0.0   0.0    0.0   -0.0    0.0  
car       0.0   0.0   -0.0   -0.0    0.0  
I         0.0  -0.0   -0.0    0.0   -0.0  
eat      -0.0   0.0    0.0    0.0   -0.0  
day       0.0  -0.0   -0.0   -0.0    0.0

In [92]:
# we can just remove the zeroos columns
TSVD_df = pd.DataFrame(lowdim_dense_matrix[:,:5], index=vocab,columns=dim_names[:5])
display(TSVD_df)

dim0      dim1      dim2      dim3      dim4
like    -0.563652  1.497387 -0.423378  0.119592 -0.713812
an      -3.590090 -0.398018 -1.152087  1.707900  0.249389
to      -0.225862  1.595626  0.189236 -0.058551 -0.175054
my      -0.609500  2.030081 -0.494240  0.142318  0.746323
driving -0.590864  1.794945 -0.501255  0.146783 -0.461930
apple   -2.011314 -0.236121 -0.338984  0.457370 -0.054726
orange  -2.011314 -0.236121 -0.338984  0.457370 -0.054726
work    -0.154674  1.260900 -0.017046  0.009616 -0.839717
every   -3.026532 -0.659660  1.762966  0.750429 -0.232352
car     -0.293771  1.874864  0.363469 -0.114090  0.852037
I       -2.620567  0.832339  2.084192 -0.552119  0.073469
eat     -3.081751 -0.211562 -1.373831 -1.499121 -0.042218
day     -2.363795 -0.593157 -0.158542 -1.849308  0.044472